In [15]:
from google.colab import files
uploaded=files.upload()
print(uploaded)

Saving timing_performance_dataset (1).csv to timing_performance_dataset (1).csv
{'timing_performance_dataset (1).csv': b'Setup_Time_Violations,Hold_Time_Violations,Critical_Path_Delay,Propagation_Delay,Clock_Skew,Clock_Domain_Crossings,Fan_In_Count,Fan_Out_Count,Combinational_Logic_Levels,Pipeline_Stages,Register_to_Register_Path_Length,Multiplexer_Depth,Gate_Count,Logic_Optimization_Ratio,Resource_Utilization,Retiming_Factor,Clock_Gating_Ratio,Combinational_Depth,Timing_Violation_Probability\n16,44,2.104520834561201,2.6830553558582615,0.39163815761426346,18,5,13,17,7,14,5,3605,0.6577673213529598,0.49689045952237243,0.583515677741571,0.040240149550669235,1,0.2653927661263312\n32,30,3.3652524240227613,2.3405943060183323,0.25917399660414026,13,3,22,15,6,11,8,1521,0.9825331135918214,0.22683953362511441,0.6283159293466986,0.6382823644251502,16,0.286643200079007\n43,1,2.436272411084385,2.0625450336574653,0.27831352235129125,1,33,34,14,8,6,1,4593,0.9725180410685264,0.4229224735707723,0.76062

In [17]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import os

# Load dataset
file_path = "timing_performance_dataset.csv"  # Replace with your actual file path


if not os.path.exists(file_path):
    print(f"Error: The file '{file_path}' was not found. Please provide the correct file path.")
    file_path = input("Enter the correct file path: ").strip()
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Error: The file '{file_path}' was still not found. Check the file location and try again.")

df = pd.read_csv(file_path)

# Define target variable
target_column = "Critical_Path_Delay"  # Adjust as needed

if target_column not in df.columns:
    raise ValueError(f"Error: The target column '{target_column}' is not found in the dataset. Available columns: {df.columns.tolist()}")

# Prepare dataset
X = df.drop(columns=[target_column])
y = df[target_column]

# Handle missing values
X.fillna(X.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define Neural Network model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile model
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred_nn = model.predict(X_test).flatten()

mae_nn = mean_absolute_error(y_test, y_pred_nn)
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print(f"Neural Network Performance:\nMAE: {mae_nn}\nMSE: {mse_nn}\nR2 Score: {r2_nn}")

Error: The file 'timing_performance_dataset.csv' was not found. Please provide the correct file path.
Enter the correct file path: timing_performance_dataset (1).csv


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 1.8328 - mae: 1.8328 - val_loss: 1.1100 - val_mae: 1.1100
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2537 - mae: 1.2537 - val_loss: 1.1441 - val_mae: 1.1441
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1323 - mae: 1.1323 - val_loss: 1.0486 - val_mae: 1.0486
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.1357 - mae: 1.1357 - val_loss: 1.0736 - val_mae: 1.0736
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1016 - mae: 1.1016 - val_loss: 1.0531 - val_mae: 1.0531
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0669 - mae: 1.0669 - val_loss: 1.0565 - val_mae: 1.0565
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0311 - mae: 1.0311 - val_loss: 1.0436 - val_mae: 1.0436
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0296 - mae: 1.0296 - val_loss: 1.0569 - val_mae: 1.0569
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.99

In [19]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np
from google.colab import files

# Upload dataset manually if not using Google Drive
uploaded = files.upload()

# Load dataset
file_path = "timing_performance_dataset.csv"  # Ensure correct file name
df = pd.read_csv(file_path)

# Define target variable
target_column = "Critical_Path_Delay"  # Adjust if needed
if target_column not in df.columns:
    raise ValueError(f"Error: Target column '{target_column}' not found. Available columns: {df.columns.tolist()}")

# Prepare dataset
X = df.drop(columns=[target_column])
y = df[target_column]

# Handle missing values
X.fillna(X.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-Fold Cross Validation (k=5)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores, mse_scores, r2_scores = [], [], []

for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Define Neural Network model
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)  # Output layer for regression
    ])

    # Compile model
    model.compile(optimizer='adam', loss='mae', metrics=['mae'])

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)  # Reduce epochs for faster training

    # Evaluate the model
    y_pred_nn = model.predict(X_test).flatten()

    mae_scores.append(mean_absolute_error(y_test, y_pred_nn))
    mse_scores.append(mean_squared_error(y_test, y_pred_nn))
    r2_scores.append(r2_score(y_test, y_pred_nn))

# Print cross-validation results
print(f"Average MAE: {np.mean(mae_scores):.4f}")
print(f"Average MSE: {np.mean(mse_scores):.4f}")
print(f"Average R2 Score: {np.mean(r2_scores):.4f}")


Saving timing_performance_dataset.csv to timing_performance_dataset.csv


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Average MAE: 1.3242
Average MSE: 2.6219
Average R2 Score: -0.5736
